In [ ]:
import json
import tiktoken
import re
import os

directory = "./raw_threads" #<---EDIT this to dir containing JSON thread files


# Token Counter
def get_tokens(string):
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(string))
    return num_tokens

# Function to split messages into 4000 character chunks
def split_messages(raw_history, string_history, link, start_time, timestamp):
    chunk_of_history = ""
    chunked_history = []

    for m in raw_history:
        name = m['name']
        msg = m["message"]
        chunk_of_history += f'{name}: {msg} \n'
        if len(chunk_of_history) > 4000:
            token_count = get_tokens(chunk_of_history)
            chunked_history.append({"is_split": True, "link": link, "messages": chunk_of_history, "full_thread": string_history, "time": start_time, "timestamp": timestamp, "token_count": token_count})
            chunk_of_history = ""
    
    return chunked_history


# Iterate over all JSON files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".json"):
        filepath = os.path.join(directory, filename)

        # Step 1: Load data
        with open(filepath, 'r') as f:
            data = json.load(f)

        final =[]
        for i in data:
            string_history = ''
            print(i)
            for j in i['thread_data']:
                name = j['name']
                msg = j["message"]
                string_history += f'{name}: {msg} \n'

            if len(string_history) > 4000:
                list_of_split_msg = split_messages(i['thread_data'], string_history, i['thread_url'], i['thread_data'][0]['time'],  i['thread_data'][0]['timestamp'])
            else:
                token_count = get_tokens(string_history)
                list_of_split_msg = [{"is_split": False, "link": i['thread_url'],  "messages": string_history, "time": i['thread_data'][0]['time'], "timestamp":   i['thread_data'][0]['timestamp'], "token_count": token_count }]

            final.extend(list_of_split_msg)

        # Write results to file
        output_filename = filename.replace('.json', '_chunks.json')
        output_filepath = os.path.join("./raw_threads/chunks", output_filename)
        with open(output_filepath, 'w', encoding='utf-8') as f:
            json.dump(final, f, ensure_ascii=False, indent=4)
